In [1]:
import pandas as pd

In [3]:
import geopandas as gpd

# Load the GeoJSON file
gdf_airspace_ohare_loaded = gpd.read_file("ohare_airspace.geojson")

# Optional: Preview the datag
gdf_airspace_ohare_loaded


,airspace_name,min_altitude,max_altitude,elevation,geometry
0,CHICAGO CLASS B AREA A,3048.00,3048.00,3043.478261,"POLYGON Z ((-87.92530 42.06970 3048.00000, -87..."
1,CHICAGO CLASS B AREA B,3048.00,3048.00,3043.478261,"POLYGON Z ((-87.79310 41.98750 3048.00000, -87..."
2,CHICAGO CLASS B AREA C,3048.00,3048.00,3043.478261,"POLYGON Z ((-87.90470 42.23760 3048.00000, -87..."
3,CHICAGO CLASS B AREA D,3048.00,3048.00,3043.478261,"POLYGON Z ((-88.16470 41.83000 3048.00000, -88..."
4,CHICAGO CLASS B AREA E,3048.00,3048.00,3043.478261,"POLYGON Z ((-87.42280 41.77780 3048.00000, -87..."
5,CHICAGO CLASS B AREA G,3048.00,3048.00,3043.478261,"POLYGON Z ((-88.01280 42.13360 3048.00000, -88..."
6,CHICAGO CLASS B AREA H,3048.00,3048.00,3043.478261,"POLYGON Z ((-87.80000 42.13500 3048.00000, -87..."
7,CHICAGO CLASS C,1097.28,1097.28,260.869565,"POLYGON Z ((-87.75250 41.86940 1097.28000, -87..."
8,CHICAGO CLASS C,1097.28,1097.28,260.869565,"POLYGON Z ((-87.55920 41.86920 1097.28000, -87..."
9,CHICAGO CLASS C,1097.28,1097.28,260.869565,"POLYGON Z ((-87.53500 41.74970 1097.28000, -87..."


In [13]:
# Get bounding box of airspace polygons
minx, miny, maxx, maxy = gdf_airspace.total_bounds
print(f"Airspace Bounds:\nLongitude: {minx} to {maxx}\nLatitude: {miny} to {maxy}")

# Extract lat/lon from geometry
gdf_ohare["longitude"] = gdf_ohare.geometry.x
gdf_ohare["latitude"] = gdf_ohare.geometry.y

# Check which points fall within the map bounds
in_bounds = gdf_ohare[
    (gdf_ohare["longitude"] >= minx) &
    (gdf_ohare["longitude"] <= maxx) &
    (gdf_ohare["latitude"] >= miny) &
    (gdf_ohare["latitude"] <= maxy)
]

print(f"Total points: {len(gdf_ohare)}")
print(f"Points in map view: {len(in_bounds)}")


Airspace Bounds:
Longitude: -89.0613 to -86.6667
Latitude: 40.75 to 43.542


AttributeError: 'Series' object has no attribute 'x'

In [18]:
gdf_ohare.columns

Index(['Unnamed: 0', 'date_of_sighting', 'state', 'city', 'summary',
       'drone_latitude', 'drone_longitude', 'drone_altitude_ft',
       'Facility Name', 'Facility Type', 'Time', 'Time_UTC', 'geometry',
       'index_right', 'airspace_name', 'min_altitude', 'max_altitude',
       'longitude', 'latitude'],
      dtype='object')

In [50]:
import pydeck as pdk
import geopandas as gpd
import pandas as pd

# === Load 3D airspace polygons ===
gdf_airspace = gpd.read_file("ohare_airspace.geojson")
gdf_airspace = gdf_airspace.explode(index_parts=False)

# Convert geometry to coordinates list (with Z)
def polygon_to_coords(poly):
    return [[list(coord) for coord in poly.exterior.coords]]

gdf_airspace["coordinates"] = gdf_airspace["geometry"].apply(polygon_to_coords)

# === Load intrusion points (CSV or GeoDataFrame with lat/lon) ===
# Example intrusion data

# === Create pydeck layers ===
sighting_layer = pdk.Layer(
    "ColumnLayer",
    data=gdf_ohare_clean,
    get_position='[longitude, latitude]',
    get_elevation='alt_scaled',
    elevation_scale=1.2,  # small boost
    radius=300,
    get_fill_color='[255, 255, 0, 180]',
    elevation_range=[1, 10000],  # ensures rendering above base
    extruded=True,
    pickable=True,
    auto_highlight=True
)


# Airspace polygons (3D extruded)
polygon_layer = pdk.Layer(
    "PolygonLayer",
    gdf_airspace,
    get_polygon="coordinates",
    get_fill_color="[200, 30, 0, 100]",
    get_elevation="elevation",
    elevation_scale=1,
    extruded=True,
    pickable=True,
    auto_highlight=True,
)

# Departure Path (blue)
departure_layer = pdk.Layer(
    "PathLayer",
    data=[{
        "path": df_flight_dept[["longitude", "latitude", "alt_scaled"]].values.tolist(),
        "name": "Departure Path"
    }],
    get_path="path",
    get_color=[0, 0, 255],  # blue
    width_scale=20,
    width_min_pixels=2,
    pickable=True
)

# Arrival Path (green)
arrival_layer = pdk.Layer(
    "PathLayer",
    data=[{
        "path": df_flight_arrivals[["longitude", "latitude", "alt_scaled"]].values.tolist(),
        "name": "Arrival Path"
    }],
    get_path="path",
    get_color=[0, 255, 0],  # green
    width_scale=20,
    width_min_pixels=2,
    pickable=True
)



# === View state (O'Hare coordinates) ===
view_state = pdk.ViewState(
    latitude=41.9742,
    longitude=-87.9073,
    zoom=9,
    pitch=60,
    bearing=0
)

# === Render map ===
r = pdk.Deck(
    layers=[
        polygon_layer,      # base layer, rendered first
        sighting_layer,     # drone sightings above polygons
        departure_layer,    # flight paths above everything
        arrival_layer
    ],
    initial_view_state=view_state,
    tooltip={
        "html": """
            {% if name %}
                <b>{name}</b><br/>
            {% endif %}

            {% if airspace_name %}
                <b>Airspace:</b> {airspace_name}<br/>
                <b>Altitude Range:</b> {min_altitude}–{max_altitude} ft<br/><br/>
            {% endif %}

            {% if summary %}
                <b>Drone Sighting</b><br/>
                <b>Date:</b> {date_of_sighting}<br/>
                <b>Time (UTC):</b> {Time_UTC}<br/>
                <b>City:</b> {city}<br/>
                <b>Summary:</b> {summary}<br/>
                <b>Drone Altitude:</b> {drone_altitude_ft} ft<br/>
            {% endif %}
        """,
        "style": {
            "backgroundColor": "black",
            "color": "white",
            "fontSize": "12px",
            "maxWidth": "300px"
        }
    }
)



# Export to HTML
r.to_html("airspace_intrusions_3d_w_tracks.html")


In [62]:
import pydeck as pdk
import geopandas as gpd
import pandas as pd

# === Load 3D airspace polygons ===
gdf_airspace = gpd.read_file("ohare_airspace.geojson")
gdf_airspace = gdf_airspace.explode(index_parts=False)

# Convert geometry to coordinates list (with Z)
def polygon_to_coords(poly):
    return [[list(coord) for coord in poly.exterior.coords]]

gdf_airspace["coordinates"] = gdf_airspace["geometry"].apply(polygon_to_coords)

# === Load intrusion points (CSV or GeoDataFrame with lat/lon) ===
# Example intrusion data

# === Create pydeck layers ===
sighting_layer = pdk.Layer(
    "ColumnLayer",
    data=gdf_ohare_clean,
    get_position='[longitude, latitude]',
    get_elevation='alt_scaled',
    elevation_scale=1,  # small boost
    radius=50,
    get_fill_color='[255, 255, 0, 180]',
    elevation_range=[1, 10000],  # ensures rendering above base
    extruded=True,
    pickable=True,
    auto_highlight=True
)


# Airspace polygons (3D extruded)
polygon_layer = pdk.Layer(
    "PolygonLayer",
    gdf_airspace,
    get_polygon="coordinates",
    get_fill_color="[200, 30, 0, 100]",
    get_elevation="elevation",
    elevation_scale=1,
    extruded=True,
    pickable=True,
    auto_highlight=True,
)

# Departure Path (blue)
departure_layer = pdk.Layer(
    "PathLayer",
    data=[{
        "path": df_flight_dept[["longitude", "latitude", "altitude"]].values.tolist(),
        "name": "Departure Path"
    }],
    get_path="path",
    get_color=[0, 0, 255],  # blue
    width_scale=20,
    width_min_pixels=2,
    pickable=True
)

# Arrival Path (green)
arrival_layer = pdk.Layer(
    "PathLayer",
    data=[{
        "path": df_flight_arrivals[["longitude", "latitude", "altitude"]].values.tolist(),
        "name": "Arrival Path"
    }],
    get_path="path",
    get_color=[0, 255, 0],  # green
    width_scale=20,
    width_min_pixels=2,
    pickable=True
)



# === View state (O'Hare coordinates) ===
view_state = pdk.ViewState(
    latitude=41.9742,
    longitude=-87.9073,
    zoom=9,
    pitch=60,
    bearing=0
)

# === Render map ===
r = pdk.Deck(
    layers=[      # base layer, rendered first
        sighting_layer,     # drone sightings above polygons
        departure_layer,    # flight paths above everything
        arrival_layer
    ],
    initial_view_state=view_state,
    tooltip={
        "html": """
            {% if name %}
                <b>{name}</b><br/>
            {% endif %}

            {% if airspace_name %}
                <b>Airspace:</b> {airspace_name}<br/>
                <b>Altitude Range:</b> {min_altitude}–{max_altitude} ft<br/><br/>
            {% endif %}

            {% if summary %}
                <b>Drone Sighting</b><br/>
                <b>Date:</b> {date_of_sighting}<br/>
                <b>Time (UTC):</b> {Time_UTC}<br/>
                <b>City:</b> {city}<br/>
                <b>Summary:</b> {summary}<br/>
                <b>Drone Altitude:</b> {drone_altitude_ft} ft<br/>
            {% endif %}
        """,
        "style": {
            "backgroundColor": "black",
            "color": "white",
            "fontSize": "12px",
            "maxWidth": "300px",
        },
    },
)



# Export to HTML
r.to_html("airspace_intrusions_3d_w_tracks_2.html")


In [64]:
from shapely.geometry import LineString, Point
import geopandas as gpd

# Step 1: Create LineStrings for flight paths
line_arrival = LineString(df_flight_arrivals[["longitude", "latitude"]].values)
line_departure = LineString(df_flight_dept[["longitude", "latitude"]].values)

# Step 2: Convert everything to GeoDataFrames
gdf_intrusions = gpd.GeoDataFrame(
    gdf_ohare_clean.copy(),
    geometry=gpd.points_from_xy(gdf_ohare_clean.longitude, gdf_ohare_clean.latitude),
    crs="EPSG:4326"
)

gdf_arrival = gpd.GeoDataFrame(geometry=[line_arrival], crs="EPSG:4326")
gdf_departure = gpd.GeoDataFrame(geometry=[line_departure], crs="EPSG:4326")

# Step 3: Project all to UTM Zone 16N (Chicago region, in meters)
gdf_intrusions_utm = gdf_intrusions.to_crs("EPSG:32616")
line_arrival_utm = gdf_arrival.to_crs("EPSG:32616").geometry.iloc[0]
line_departure_utm = gdf_departure.to_crs("EPSG:32616").geometry.iloc[0]

# Step 4: Calculate minimum distance from each drone point to both flight tracks
gdf_intrusions_utm["dist_to_arrival_m"] = gdf_intrusions_utm.distance(line_arrival_utm)
gdf_intrusions_utm["dist_to_departure_m"] = gdf_intrusions_utm.distance(line_departure_utm)
gdf_intrusions_utm["min_distance_m"] = gdf_intrusions_utm[["dist_to_arrival_m", "dist_to_departure_m"]].min(axis=1)

# Step 5: Get average distance
average_distance = gdf_intrusions_utm["min_distance_m"].mean()
print(f"Average distance from drone intrusions to flight paths: {average_distance:.2f} meters")


Average distance from drone intrusions to flight paths: 8125.12 meters


/Users/vishalivallioor/anaconda3/lib/python3.10/site-packages/shapely/measurement.py:74: RuntimeWarning: invalid value encountered in distance
  return lib.distance(a, b, **kwargs)
/Users/vishalivallioor/anaconda3/lib/python3.10/site-packages/shapely/measurement.py:74: RuntimeWarning: invalid value encountered in distance
  return lib.distance(a, b, **kwargs)


In [66]:
closest_distance = gdf_intrusions_utm["min_distance_m"].min()
closest_distance

89.04964668321422

In [48]:
df_flight_dept["alt_scaled"] = df_flight_dept["altitude"] / 10
df_flight_arrivals["alt_scaled"] = df_flight_arrivals["altitude"] / 10

In [23]:
import pandas as pd
import geopandas as gpd
from shapely import wkt
from shapely.geometry import Point

# Step 1: Load CSV
gdf_ohare = pd.read_csv("ohare2.csv")

# Create geometry from drone_latitude and drone_longitude
gdf_ohare["geometry"] = gdf_ohare.apply(
    lambda row: Point(row["drone_longitude"], row["drone_latitude"]), axis=1
)

# Convert to GeoDataFrame with correct CRS
gdf_ohare = gpd.GeoDataFrame(gdf_ohare, geometry="geometry", crs="EPSG:4326")

# (Optional but good): extract lat/lon for plotting
gdf_ohare["longitude"] = gdf_ohare.geometry.x
gdf_ohare["latitude"] = gdf_ohare.geometry.y


In [27]:
# Drop any rows that can't be visualized
gdf_ohare_clean = gdf_ohare.dropna(subset=["longitude", "latitude", "alt_scaled"])
print(f"Total after cleaning: {len(gdf_ohare_clean)}")  # should be close to 100


Total after cleaning: 165


In [ ]:
import pydeck as pdk
import geopandas as gpd
import pandas as pd

# === Load 3D airspace polygons ===
gdf_airspace = gpd.read_file("ohare_airspace.geojson")
gdf_airspace = gdf_airspace.explode(index_parts=False)

# Convert geometry to coordinates list (with Z)
def polygon_to_coords(poly):
    return [[list(coord) for coord in poly.exterior.coords]]

gdf_airspace["coordinates"] = gdf_airspace["geometry"].apply(polygon_to_coords)

# === Create pydeck layers ===

# Airspace polygons (3D extruded)
polygon_layer = pdk.Layer(
    "PolygonLayer",
    gdf_airspace,
    get_polygon="coordinates",
    get_fill_color="[200, 30, 0, 100]",
    get_elevation="elevation",
    elevation_scale=1,
    extruded=True,
    pickable=True,
    auto_highlight=True,
)

# Drone intrusion dots
dot_layer = pdk.Layer(
    "ScatterplotLayer",
    data=df_intrusions,
    get_position='[longitude, latitude]',
    get_color='[0, 0, 255, 160]',
    get_radius=300,
    pickable=True,
)

# === View state (O'Hare coordinates) ===
view_state = pdk.ViewState(
    latitude=41.9742,
    longitude=-87.9073,
    zoom=9,
    pitch=60,
    bearing=0
)

# === Render map ===
r = pdk.Deck(
    layers=[polygon_layer, dot_layer],
    initial_view_state=view_state,
    tooltip={"text": "{label}"}
)

# Export to HTML
r.to_html("airspace_intrusions_3d.html")


In [8]:
print(df_intrusions.head())


   latitude  longitude  altitude    label
0   41.9802   -87.9085       500  Drone A
1   42.0011   -87.9320      1200  Drone B
2   41.9705   -87.9001       800  Drone C


In [53]:
import xml.etree.ElementTree as ET
import pandas as pd

# Define namespaces
ns = {
    'kml': 'http://www.opengis.net/kml/2.2',
    'gx': 'http://www.google.com/kml/ext/2.2'
}

# Parse KML
tree = ET.parse("D-AALG-track-EGM96.kml")
root = tree.getroot()

# Extract <when> timestamps and <gx:coord> positions
when_elements = root.findall(".//gx:Track/gx:when", ns)
coord_elements = root.findall(".//gx:Track/gx:coord", ns)

# Convert text to lists
timestamps = [elem.text for elem in when_elements]
coords = [list(map(float, elem.text.strip().split())) for elem in coord_elements]

# Create DataFrame
df_flight_arrivals = pd.DataFrame(coords, columns=["longitude", "latitude", "altitude"])



In [54]:
import xml.etree.ElementTree as ET
import pandas as pd

# Define namespaces
ns = {
    'kml': 'http://www.opengis.net/kml/2.2',
    'gx': 'http://www.google.com/kml/ext/2.2'
}

# Parse KML
tree = ET.parse("N580UP-track-EGM96.kml")
root = tree.getroot()

# Extract <when> timestamps and <gx:coord> positions
when_elements = root.findall(".//gx:Track/gx:when", ns)
coord_elements = root.findall(".//gx:Track/gx:coord", ns)

# Convert text to lists
timestamps = [elem.text for elem in when_elements]
coords = [list(map(float, elem.text.strip().split())) for elem in coord_elements]

# Create DataFrame
df_flight_dept = pd.DataFrame(coords, columns=["longitude", "latitude", "altitude"])


In [46]:
df_flight_dept

,longitude,latitude,altitude
0,-87.908063,41.961594,0.0
1,-87.908063,41.961594,0.0
2,-87.908072,41.961765,0.0
3,-87.908063,41.961803,0.0
4,-87.908072,41.961811,0.0
...,...,...,...
379,-87.683105,40.413071,8758.0
380,-87.676017,40.375920,8758.0
381,-87.668901,40.339004,8758.0
382,-87.663595,40.311445,8758.0


In [47]:
df_flight_arrivals

,longitude,latitude,altitude
0,-79.631260,43.679488,0.0
1,-79.631260,43.679488,0.0
2,-79.631195,43.679429,0.0
3,-79.631048,43.679278,0.0
4,-79.631030,43.679260,0.0
...,...,...,...
780,-87.914195,41.961559,0.0
781,-87.914195,41.961559,0.0
782,-87.914195,41.961501,0.0
783,-87.914195,41.961361,0.0
